Хранение расписания тренировок с детализацией упражнений по каждой тренировке по каждому участнику.

### Создаём бд

In [3]:
import sqlite3
import string
import random

In [27]:
con = sqlite3.connect("timetable.db")
cursor = con.cursor()

In [299]:
#cursor.execute("DROP TABLE users")
cursor.execute("DROP TABLE trainings")
#cursor.execute("DROP TABLE schedule")

In [140]:
cursor.execute("CREATE TABLE users (u_id integer, name text,  PRimary key (u_id))")

In [300]:
cursor.execute("CREATE TABLE trainings (t_id integer, complex text,  PRimary key (t_id))")

In [153]:
cursor.execute("CREATE TABLE schedule (s_id integer, date text, u_id integer, t_id integer,  PRimary key (s_id))")

### Заполняем бд

In [143]:
def fill_users():
    for i in range(1, 101):
        name = ''.join(random.choice(string.ascii_lowercase) for _ in range(10)).capitalize()
        cursor.execute('INSERT INTO users (u_id, name) VALUES ({},\"{}\")'.format(str(i), name))
    con.commit()
    return

In [144]:
fill_users()

In [145]:
train_set = ['бег 10 минут, 20 подтягиваний, 30 приседаний', 'планка 2 минуты, скакалка 500 раз, 20 выпадов',
             'йога 1 час', '4 бассейна кролем, 4 бассейна на спине, 16 бассейнов вольным стилем']

In [301]:
def fill_trainings(train_set):
    for i in range(1, len(train_set)+1):
        cursor.execute('INSERT INTO trainings (t_id, complex) VALUES ({},\"{}\")'.format(str(i), train_set[i-1]))
    con.commit()
    return

In [305]:
fill_trainings(train_set)

In [151]:
data = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']

In [155]:
def fill_schedule(data):
    day = random.choice(data)
    num_people = random.randint(1, 15)
    num_train = random.randint(1, 4)
    for i in range(1, 101):
        day = random.choice(data)
        num_people = random.randint(1, 15)
        num_train = random.randint(1, 4)
        cursor.execute('INSERT INTO schedule (s_id, date, u_id , t_id) VALUES ({},\"{}\", \"{}\", \"{}\")'.format(
         str(i), day, random.sample(range(1, 101), num_people), num_train
     ))
    con.commit()
    return

In [156]:
fill_schedule(data)

In [ ]:
file = 'timetable' + '.db'

### Добавление новых данных

In [389]:
def check_name(name):
    n = cursor.execute("SELECT name FROM users WHERE name = '{}'".format(name)).fetchall()
    if n != []:
        return (str(n[0]).strip(',)('))
    else:
        return None

In [382]:
def add_name(name, timetable):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    
    nums = cursor.execute("SELECT u_id FROM users").fetchall()
    i = len(nums)+1
    cursor.execute('INSERT INTO users (u_id, name) VALUES ({},\"{}\")'.format(str(i), name))
    print('Новое имя добавлено в базу данных.')
    con.commit()
    return

In [381]:
def add_complex(train, timetable):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    
    nums = cursor.execute("SELECT t_id FROM trainings").fetchall()
    i = len(nums)+1
    cursor.execute('INSERT INTO trainings (t_id, complex) VALUES ({},\"{}\")'.format(str(i), train))
    print('Новый тип тренировок добавлен в базу данных.')
    con.commit()
    return

In [380]:
def add_to_schedule(people, train, dates, timetable):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    
    names = cursor.execute("SELECT * FROM users").fetchall()
    u_ids = []
    for name in people:
        for i in range(1, len(names)):
            if name== names[i][1]:
                u_ids.append(names[i][0])
    i = len(cursor.execute("SELECT s_id FROM schedule").fetchall())+1
    
    for day in dates:
        d = day.capitalize()
        cursor.execute('INSERT INTO schedule (s_id, date, u_id , t_id) VALUES ({},\"{}\", \"{}\", \"{}\")'.format(
            str(i), d, u_ids, train
        ))
        i+=1
    print ('Новая тренировка добавлена в базу данных')
    con.commit()
    return

In [387]:
def add_new_data():
    file = 'timetable' + '.db'
    
    type_add = input('Если вы хотите добавить нового участника, введите 1.' + '\n' + 
                     'Если вы хотите добавить новый тип тренировки, введите 2.'  + '\n' + 
                     'Если вы хотите занести новую тренировку в расписание, введите 3.')
    if type_add == str(1):
        target_name = input('Введите имя человека.')
        if check_name(target_name) is not None:
            print('Такой человек уже есть в базе.')
        else:
            add_name(target_name, file)
            
    if type_add == str(2):
        trainings = cursor.execute("SELECT complex FROM trainings").fetchall()
        for i in range(0, len(trainings)):
            print(trainings[i])
        target_complex = input('Посмотрите внимательно, нет ли уже в базе тренировки такого типа.'  + '\n' + 
        'Если есть, нажмите клавишу Enter. Если нет, введите составляющие упражнения через запятую.')
        if target_complex != '':
            add_complex(target_complex, file)
            
    if type_add == str(3):
        trainings = cursor.execute("SELECT * FROM trainings").fetchall()
        for i in range(0, len(trainings)):
            print(trainings[i])
        new_train = input('Введите порядковый номер тренировки.')
        new_people = input ('Введите имена людей, которые будут посещать тренировку, через запятую.')
        new_data = input ('Введите день недели, в который будет проходить тренировка.'  + '\n' + 
                         'Если тренировка будет проходить несколько раз в неделю, введите дни через запятую.')
        new_people = new_people.split(', ')
        for name in new_people:
            if check_name(name) is None:
                add_name(name)
        add_to_schedule(new_people, new_train, new_data.split(','), file)
    
    return

In [ ]:
add_new_data()

### Поиск по дб

In [71]:
#выводит имена людей в алфавитном порядке
def alph_names(timetable):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    cursor.execute("SELECT name, u_id FROM USERS ORDER BY name")
    alph_names = []
    for name in cursor.fetchall():
        alph_names.append(name[0])
    return(alph_names)

In [429]:
#выводит все тренировки и их участников в определённый день
def get_day_train(timetable, day):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    days = cursor.execute("SELECT date, u_id, t_id FROM schedule WHERE date='{}'".format(day)).fetchall()
    if len(days) == 0:
        return('В этот день тренировок нет.')
    day_train = []
    for t in days:
        day_train.append(t)
    return(day_train)

In [418]:
#выводит дни, в которые есть тренировка
def when_training(timetable, train):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    trains = cursor.execute("SELECT s_id, date FROM schedule WHERE t_id='{}'".format(train)).fetchall()
    if len(trains) == 0:
        return('Такого типа тренировок не существует.')
    trainings = []
    for t in trains:
        trainings.append(t)
    return(trainings)

In [393]:
#выводит список тренировок по каждому участнику
def train_for_each(timetable):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    
    people = []
    for p in cursor.execute("SELECT u_id FROM users").fetchall():
        people.append(str(p).strip(')(,'))

    data = {}

    people_groups = cursor.execute("SELECT u_id FROM schedule").fetchall()
    for p in people:
        data[p]=[]
        for group in people_groups:
            if p in group[0].strip('[]').split(', '):
                data[p].append(cursor.execute("SELECT t_id, date FROM schedule WHERE u_id='{}'".format(group[0])).fetchall())
                
    names = cursor.execute("SELECT * FROM users").fetchall()
    data_with_names = {}
    for key, value in data.items():
        for name in names:
            if key == str(name[0]):
                d = data[key]
                data_with_names[name[1]] = d
    return data_with_names

In [396]:
#выводит список тренировок и дат для одного человека.
def personal_train(timetable, person):
    con = sqlite3.connect(timetable)
    cursor = con.cursor()
    num = cursor.execute("SELECT u_id FROM users WHERE name = '{}'".format(person)).fetchall()
    if len(num) == 0:
        return('Такого человека нет в базе.')
    num = str(num[0]).strip(',)(')
    
    people_groups = cursor.execute("SELECT u_id FROM schedule").fetchall()
    
    data = {}
    data[person] = []
    for group in people_groups:
        if a in group[0].strip('[]').split(', '):
            data[person].append(cursor.execute("SELECT t_id, date FROM schedule WHERE u_id='{}'".format(group[0])).fetchall())
    return (data)

In [400]:
def search(file):
    type_search = input('Введите порядковый номер поиска, который вы хотите совершить.' + '\n' +
                       '1. Имена людей в алфавитном порядке.'+ '\n' +
                       '2. Все тренировки и участники в определённый день недели.'+ '\n' +
                       '3. Все дни, когда есть определённый тип тренировок.'+ '\n' +
                       '4. Список всех тренировок у каждого участника.'+ '\n' +
                       '5. Расписание тренировок у определённого человека.')
    
    if type_search == str(1):
        print(alph_names(file))
    elif type_search == str(2):
        day = input('Введите интересующий вас день недели.')
        day = day.capitalize()
        print(get_day_train(file, day))
    elif type_search == str(3):
        trainings = cursor.execute("SELECT * FROM trainings").fetchall()
        for i in range(0, len(trainings)):
            print(trainings[i])
        train = input('Введите порядковый номер тренировки.')
        print(when_training(file, train))
    elif type_search == str(4):
        print(train_for_each(file))
    elif type_search == str(5):
        name = input('Введите имя интересующего вас человека.')
        print(personal_train(file, name))
    else:
        return
    return

In [442]:
search(file)

Введите порядковый номер поиска, который вы хотите совершить.
1. Имена людей в алфавитном порядке.
2. Все тренировки и участники в определённый день недели.
3. Все дни, когда есть определённый тип тренировок.
4. Список всех тренировок у каждого участника.
5. Расписание тренировок у определённого человека.5
Введите имя интересующего вас человека.Костя
Такого человека нет в базе.


**Примеры поиска.**

In [439]:
al = alph_names(file)
for i in range(10):
    print (al[i])

Agaxnivrep
Agaxnivrep
Agaxnivrep
Anwjhrdozb
Aujqhvaonm
Baofnjbeww
Bnpcjdoltj
Bqwlpakmdm
Bvjfnccpbo
Cefjvbijcw


In [420]:
get_day_train(file, 'Понедельник')

[('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',),
 ('Понедельник',)]

In [78]:
when_training(file, '4')

[(2, 'Пятница'),
 (5, 'Четверг'),
 (9, 'Суббота'),
 (11, 'Четверг'),
 (12, 'Вторник'),
 (13, 'Суббота'),
 (16, 'Воскресенье'),
 (18, 'Вторник'),
 (23, 'Суббота'),
 (25, 'Среда'),
 (27, 'Воскресенье'),
 (32, 'Суббота'),
 (33, 'Среда'),
 (38, 'Воскресенье'),
 (39, 'Вторник'),
 (44, 'Вторник'),
 (48, 'Среда'),
 (52, 'Воскресенье'),
 (55, 'Четверг'),
 (59, 'Четверг'),
 (60, 'Четверг'),
 (63, 'Пятница'),
 (65, 'Воскресенье'),
 (69, 'Четверг'),
 (72, 'Понедельник'),
 (74, 'Воскресенье'),
 (78, 'Пятница'),
 (84, 'Вторник'),
 (95, 'Пятница'),
 (96, 'Среда'),
 (98, 'Пятница')]

In [192]:
train_for_each(file)

{'Agaxnivrep': [[(3, 'Вторник')],
  [(2, 'Среда')],
  [(2, 'Среда')],
  [(4, 'Четверг')],
  [(4, 'Четверг')],
  [(2, 'Суббота')],
  [(2, 'Воскресенье')]],
 'Anwjhrdozb': [[(3, 'Вторник')],
  [(3, 'Воскресенье')],
  [(1, 'Среда')],
  [(2, 'Понедельник')],
  [(4, 'Понедельник')],
  [(2, 'Вторник')]],
 'Aujqhvaonm': [[(4, 'Четверг')],
  [(4, 'Среда')],
  [(3, 'Среда')],
  [(3, 'Воскресенье')],
  [(2, 'Суббота')],
  [(1, 'Вторник')],
  [(1, 'Воскресенье')]],
 'Baofnjbeww': [[(3, 'Вторник')],
  [(1, 'Воскресенье')],
  [(4, 'Воскресенье')],
  [(3, 'Четверг')],
  [(3, 'Воскресенье')],
  [(2, 'Суббота')],
  [(4, 'Четверг')],
  [(3, 'Воскресенье')],
  [(3, 'Суббота')]],
 'Bnpcjdoltj': [[(4, 'Понедельник')],
  [(3, 'Воскресенье')],
  [(4, 'Пятница')],
  [(4, 'Четверг')],
  [(1, 'Воскресенье')],
  [(4, 'Пятница')],
  [(3, 'Среда')],
  [(1, 'Понедельник')],
  [(3, 'Воскресенье')],
  [(2, 'Пятница')],
  [(3, 'Суббота')],
  [(1, 'Четверг')],
  [(4, 'Суббота')]],
 'Bqwlpakmdm': [[(1, 'Вторник')],
  [

In [233]:
person = 'Agaxnivrep'
personal_train(file, person)

{'Agaxnivrep': [[(3, 'Вторник')],
  [(2, 'Среда')],
  [(2, 'Среда')],
  [(4, 'Четверг')],
  [(4, 'Четверг')],
  [(2, 'Суббота')],
  [(2, 'Воскресенье')]]}

In [443]:
timetable.db

NameError: name 'timetable' is not defined